In [9]:
#Importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import os

from sklearn.utils import class_weight
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50
from keras.layers import Dense, Flatten, Dropout
from keras.models import Model
%matplotlib inline

In [31]:
# define constants
img_width, img_height = 200, 200
target_size = (200,200)
batch_size = 28
epochs = 10
learning_rate = 0.001

In [2]:
# Set the paths to your training, validation, and testing folders
train_folder = "Original Images/a. Training Set"
valid_folder = "Original Images/b. Validation Set"
test_folder = "Original Images/c. Testing Set"

# Set the paths to your training, validation, and testing CSV files for labels
train_labels_file = "Groundtruths/training28.csv"
valid_labels_file = "Groundtruths/validation28.csv"
test_labels_file = "Groundtruths/testing28.csv"

In [70]:
# Load the CSV file with image filenames and labels
train_labels_df = pd.read_csv(train_labels_file)
valid_labels_df = pd.read_csv(valid_labels_file)
test_labels_df = pd.read_csv(test_labels_file)

# Remove rows from dataframes where filenames are missing
missing_rows = [1506,1505,1744,1752,1753,1756,1762,1770,1781,1788,1792,1842,1846,1873,1878] 
train_labels_df.drop(missing_rows, inplace=True)
missing_rows = [549,550,552,560,590,596]
valid_labels_df.drop(missing_rows, inplace=True)
missing_rows = [553,590,592,564,614,617,622,624,628]
test_labels_df.drop(missing_rows, inplace=True)

# Add the .png extension to the filenames
train_labels_df['ID'] = train_labels_df['ID'].apply(lambda x: str(x) + '.png')
valid_labels_df['ID'] = valid_labels_df['ID'].apply(lambda x: str(x) + '.png')
test_labels_df['ID'] = test_labels_df['ID'].apply(lambda x: str(x) + '.png')

# Calculate the number of positive cases in each label except the ID column
label_counts = train_labels_df.iloc[:, 1:].sum(axis=0)

# Sort the labels in descending order by number of positive cases
sorted_labels = label_counts.sort_values(ascending=False)

# Reorder the columns in train_df, valid_df, and test_df using the sorted label order
train_labels_df = train_labels_df[['ID'] + list(sorted_labels.index)]
valid_labels_df = valid_labels_df[['ID'] + list(sorted_labels.index)]
test_labels_df = test_labels_df[['ID'] + list(sorted_labels.index)]

#Dropping disease risk label
train_labels_df = train_labels_df.drop('Disease_Risk', axis=1)
valid_labels_df = valid_labels_df.drop('Disease_Risk', axis=1)
test_labels_df = test_labels_df.drop('Disease_Risk', axis=1)

# Print the updated dataframe to verify the sorting
print(train_labels_df.head())
train_labels_df.columns

      ID  DR  MH  ODC  TSLN  DN  MYA  ARMD  BRVO  ODP  ...  AH  MS  EDN  RT  \
0  1.png   1   0    0     0   0    0     0     0    0  ...   0   0    0   0   
1  2.png   1   0    0     0   0    0     0     0    0  ...   0   0    0   0   
2  3.png   1   0    0     0   0    0     0     0    0  ...   0   0    0   0   
3  4.png   0   1    1     0   0    0     0     0    0  ...   0   0    0   0   
4  5.png   1   0    0     0   0    0     0     0    0  ...   0   0    0   0   

   ERM  PT  MHL  RP  TV  ST  
0    0   0    0   0   0   0  
1    0   0    0   0   0   0  
2    0   0    0   0   0   0  
3    0   0    0   0   0   0  
4    0   0    0   0   0   0  

[5 rows x 29 columns]


Index(['ID', 'DR', 'MH', 'ODC', 'TSLN', 'DN', 'MYA', 'ARMD', 'BRVO', 'ODP',
       'ODE', 'LS', 'RS', 'CSR', 'OTHER', 'CRS', 'CRVO', 'RPEC', 'AION', 'AH',
       'MS', 'EDN', 'RT', 'ERM', 'PT', 'MHL', 'RP', 'TV', 'ST'],
      dtype='object')

In [108]:
from sklearn.utils import shuffle

# Get the last 15 labels as minority labels
minority_labels = list(train_labels_df.columns[-15:])

# Create a minority_df that contains only the minority labels and the ID column
minority_df = train_labels_df[['ID'] + minority_labels]

# Shuffle the minority_df to avoid having batches of only one label
minority_df = shuffle(minority_df, random_state=42)

# Define the rescaling generator
rescale_datagen = ImageDataGenerator(rescale=1./255)

# Define ImageDataGenerator for augmentation
aug_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest')

# create class weight dictionary
class_weights = {label: (len(train_labels_df)/train_labels_df[label].sum()) for label in minority_labels}

In [110]:
# Load train dataset using flow_from_dataframe
train_generator = rescale_datagen.flow_from_dataframe(
    dataframe=train_labels_df,
    directory=train_folder,
    x_col="ID",
    y_col=train_labels_df.columns[1:],
    batch_size=batch_size,
    seed=42,
    shuffle=True,
    class_mode="raw",
    target_size=(img_height, img_width))

Found 1905 validated image filenames.


In [111]:
# Create a generator for the minority_df that applies augmentation to minority labels
minority_augmented_generator = aug_datagen.flow_from_dataframe(
    dataframe=minority_df,
    directory=train_folder,
    x_col="ID",
    y_col=train_labels_df.columns[1:],
    batch_size=batch_size,
    seed=42,
    shuffle=True,
    class_mode="raw",
    target_size=(img_height, img_width),
    class_weight=class_weights)

KeyError: "['DR', 'MH', 'ODC', 'TSLN', 'DN', 'MYA', 'ARMD', 'BRVO', 'ODP', 'ODE', 'LS', 'RS', 'CSR'] not in index"

In [91]:

)

Found 1905 validated image filenames.


In [98]:
train_labels_df['ID'] = train_labels_df['ID'].astype(str)

In [107]:
minority_tuple = tuple(labels_to_oversample)
minority_df=train_labels_df[train_labels_df[list(train_labels_df.columns)].isin(minority_tuple)].loc[:, ['ID', list(train_labels_df.columns)]]

TypeError: unhashable type: 'list'

In [99]:
# Create a new generator that applies augmentation to minority labels
minority_augmented_generator = aug_datagen.flow_from_dataframe(
    dataframe=train_labels_df[train_labels_df[list(train_labels_df.columns)].isin(labels_to_oversample.tolist())],
    directory=train_folder,
    x_col='ID',
    y_col=list(train_labels_df.columns),
    batch_size=batch_size,
    seed=42,
    shuffle=True,
    class_mode="raw",
    target_size=(img_height, img_width),
    class_weight=class_weights
)

# Combine the rescaled and augmented generators using zip
train_generator = zip(train_generator, minority_augmented_generator)

TypeError: All values in column x_col=ID must be strings.

In [84]:
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create flow generators for the validation set
valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=valid_labels_df,
    directory=valid_folder,
    x_col='ID',
    y_col=valid_labels_df.columns[1:],
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='raw',
    shuffle=False)


# Create flow generators for the testing set
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_labels_df,
    directory=test_folder,
    x_col='ID',
    y_col=valid_labels_df.columns[1:],
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='raw',
    shuffle=False)

Found 634 validated image filenames.
Found 631 validated image filenames.


In [82]:
# apply oversampling to the train generator
ros = RandomOverSampler()
X_resampled, y_resampled = ros.fit_resample(train_generator[0][0], train_generator[0][1])
train_generator = (X_resampled, y_resampled)

TypeError: '<' not supported between instances of 'int' and 'str'

Found 634 validated image filenames.
Found 631 validated image filenames.


In [ ]:
# Identify the minority classes
class_freq = labels.sum(axis=0)
minority_classes = class_freq[class_freq < (class_freq.max() / 2)].index[-15:]

# Create a dictionary of class weights to be used during training
class_weights = {}
for i, col in enumerate(labels.columns):
    if col in minority_classes:
        class_weights[i] = 'balanced'
    else:
        class_weights[i] = 1

# Create a list of file paths to the images in the dataset
filepaths = [os.path.join(image_dir_path, filename) for filename in labels.index]

# Create a generator object to load the images
train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.1,
    height_shift_range=0.1,
    fill_mode='nearest',
)
train_generator = train_datagen.flow_from_dataframe(
    dataframe=labels,
    directory=image_dir_path,
    x_col='filename',
    y_col=list(labels.columns),
    target_size=image_size,
    class_mode='raw',
    batch_size=batch_size,
    shuffle=True
)

# Create a pipeline object to oversample only the minority classes
oversampler = RandomOverSampler(sampling_strategy='minority', random_state=42)
oversampling_pipeline = Pipeline([('oversampler', oversampler)])

# Apply the oversampling pipeline to the training set
X_resampled, y_resampled = oversampling_pipeline.fit_resample(train_generator, train_generator.labels)

# Reindex the DataFrame to match the resampled order
resampled_labels = labels.iloc[X_resampled.index].reset_index(drop=True)

# Create a generator object to load the resampled images
resampled_generator = train_datagen.flow_from_dataframe(
    dataframe=resampled_labels,
    directory=image_dir_path,
    x_col='filename',
    y_col=list(resampled_labels.columns),
    target_size=image_size,
    class_mode='raw',
    batch_size=batch_size,
    shuffle=True
)

# Train the model using the resampled data and the class weights
model.fit(resampled_generator, epochs=num_epochs, class_weight=class_weights)

In [7]:
# Create oversampler
oversampler = RandomOverSampler()

# Define function to load data and apply oversampling
def load_data(folder, labels):
    # Load images and labels using ImageDataGenerator
    generator = train_datagen.flow_from_directory(
        folder,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='raw',
        shuffle=False)
    # Get labels in the correct order
    labels = labels.reindex(generator.filenames)
    # Convert labels to numpy array
    labels = labels.values
    # Apply oversampling on minority classes
    X_resampled, y_resampled = oversampler.fit_resample(generator.filepaths, labels)
    # Convert filepaths to numpy array
    X_resampled = np.array(X_resampled)
    # Convert labels to one-hot encoding
    y_resampled = keras.utils.to_categorical(y_resampled, num_classes=29)
    # Shuffle the data
    indices = np.arange(len(X_resampled))
    np.random.shuffle(indices)
    X_resampled = X_resampled[indices]
    y_resampled = y_resampled[indices]
    # Create generator for the oversampled data
    generator = train_datagen.flow_from_dataframe(
        pd.DataFrame({'filename': X_resampled, 'class': np.argmax(y_resampled, axis=1)}),
        directory=folder,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)
    return generator

In [31]:
# Define Keras image data generators for training, validation, and test sets
train_datagen = ImageDataGenerator(
        rescale=1./255, 
        rotation_range=20, 
        width_shift_range=0.1, 
        height_shift_range=0.1, 
        shear_range=0.1, 
        zoom_range=0.1, 
        horizontal_flip=True)

valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Define the class weights for the imbalanced dataset
class_weights = class_weight.compute_class_weight('balanced', np.unique(train_labels_df['label']), train_labels_df['label'])

In [ ]:
import numpy as np


# read data and labels
data = []
labels = []

# code to read the data and labels

# convert data and labels to numpy arrays
data = np.array(data)
labels = np.array(labels)

# convert labels to one-hot encoding
labels = to_categorical(labels, num_classes=28)

# split data into training, validation, and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=640, stratify=labels)
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=640, stratify=train_labels)

# compute class weights
class_weights = class_weight.compute_class_weight('balanced', np.unique(train_labels.argmax(axis=1)), train_labels.argmax(axis=1))

# create undersampler
undersample = RandomUnderSampler(sampling_strategy='auto', random_state=42)

# fit and transform undersampler on training data
train_data, train_labels = undersample.fit_resample(train_data.reshape(-1, 640, 640, 3), train_labels.argmax(axis=1))
train_labels = to_categorical(train_labels, num_classes=28)

# create oversampler
oversample = RandomOverSampler(sampling_strategy='auto', random_state=42)

# create image data generator for data augmentation
datagen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.1, horizontal_flip=True)

# fit and transform oversampler on training data
train_data, train_labels = oversample.fit_resample(train_data, train_labels)

# create batches of augmented data
train_generator = datagen.flow(train_data, train_labels, batch_size=32)

# create validation and testing generators without data augmentation
val_generator = ImageDataGenerator().flow(val_data, val_labels, batch_size=32)
test_generator = ImageDataGenerator().flow(test_data, test_labels, batch_size=32)

# create ResNet50 model and train on augmented data
# code to create and train ResNet50 model